In [2]:
import matplotlib.pyplot as plt
import numpy as np
import xarray as xr
import glob
import matplotlib
from netCDF4 import Dataset
import sys
import os

# define plot dimensions
%matplotlib inline  
plt.rcParams["figure.figsize"] = (8.,8.)

from mpl_toolkits.basemap import Basemap
import matplotlib.colors as cols
plt.rcParams["figure.figsize"] = (20,12)

### axis_font = {'fontname':'Arial', 'size':'18'}    
title_font = {'fontname':'Arial', 'size':'32', 'color':'black', 'weight':'normal'}
matplotlib.rc('xtick', labelsize=28)
matplotlib.rc('ytick', labelsize=28)

In [6]:
#path to raw ARGO file downloaded from http://sio-argo.ucsd.edu/RG_Climatology.html
#Note that script expects files named RG_ArgoClim_Temperature_2017.nc and RG_ArgoClim_Salinity_2017.nc

basePath = '/lcrc/group/acme/lvanroe'
nameOfOutputFile = 'ArgoClimatology_TS.nc'

In [33]:
ncidT = Dataset(basePath+'/RG_ArgoClim_Temperature_2017.nc','r')
ncidS = Dataset(basePath+'/RG_ArgoClim_Salinity_2017.nc','r')

lon = ncidT.variables['LONGITUDE'][:]
lat = ncidT.variables['LATITUDE'][:]

anomT = ncidT.variables['ARGO_TEMPERATURE_ANOMALY'][:,:,:,:]
meanT = ncidT.variables['ARGO_TEMPERATURE_MEAN'][:,:,:]

anomS = ncidS.variables['ARGO_SALINITY_ANOMALY'][:,:,:,:]
meanS = ncidS.variables['ARGO_SALINITY_MEAN'][:,:,:]

bmask = ncidT.variables['BATHYMETRY_MASK'][:]

depth = -ncidT.variables['PRESSURE'][:]

In [34]:
#forms monthly averages of T and S
monthV = np.zeros(12);
nx = len(lon)
ny = len(lat)
nz = len(depth)

theta = np.zeros((12,nz,ny,nx))
salinity = np.zeros((12,nz,ny,nx))

for i in range(12):
    monthV[i] = i
    theta[i,:,:,:] = anomT[i::12,:,:,:].mean(axis=0)+meanT
    salinity[i,:,:,:] = anomS[i::12,:,:,:].mean(axis=0)+meanS

In [35]:
#Mask bathymetry
bmask = np.roll(bmask,20,axis=2)
for i in range(nz):
    inds1=np.where(bmask[i,:,:] != 1.0)[0]
    inds2=np.where(bmask[i,:,:] != 1.0)[1]
    salinity[:,i,inds1,inds2] = np.nan
    theta[:,i,inds1,inds2] = np.nan

In [36]:
#need to append a few rows to have a 180 x 360 matrix
theta2 = np.zeros((12,58,180,360))
salt2 = np.zeros((12,58,180,360))

In [37]:
theta2[:,:,25:170,:] = theta
salt2[:,:,25:170,:] = salinity

theta2[:,:,:25,:] = np.nan
theta2[:,:,170:,:] = np.nan

salt2[:,:,:25,:] = np.nan
salt2[:,:,170:,:] = np.nan

In [38]:
if os.path.isfile(basePath+'/'+nameOfOutputFile):
    os.remove(basePath+'/'+nameOfOutputFile)
ncid = Dataset(basePath+'/'+nameOfOutputFile,'w')
ncid.createDimension('Time',None)
ncid.createDimension('LATITUDE',180)
ncid.createDimension('LONGITUDE',360)
ncid.createDimension('DEPTH',58)

month = ncid.createVariable('month','i4',('Time'))
lat = ncid.createVariable('LATITUDE','f8',('LATITUDE'))
lonV = ncid.createVariable('LONGITUDE','f8',('LONGITUDE'))
dep = ncid.createVariable('DEPTH','f8',('DEPTH'))
thet=ncid.createVariable('theta','f8',('Time','DEPTH','LATITUDE','LONGITUDE'))
sal=ncid.createVariable('salinity','f8',('Time','DEPTH','LATITUDE','LONGITUDE'))

lat[:] = np.arange(-89.5,90,1)
lonV[:] = lon[:]
thet[:,:,:,:] = theta2
sal[:,:,:,:] = salt2
dep[:] = depth[:]
month[:] = monthV[:]

ncid.close()